In [ ]:
!pip install spacy nltk pandas tqdm
# Attempt to download a Chinese model if available (optional)
!python -m spacy download zh_core_web_sm || true
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
import nltk
from tqdm import tqdm
# For Chinese we fallback to a blank Chinese pipeline if model not present
try:
    nlp = spacy.load("zh_core_web_sm")
except Exception:
    nlp = spacy.blank("zh")
import inflect

In [ ]:
# Initialize the inflect engine (kept for API compatibility; Chinese doesn't use articles)
p = inflect.engine()

In [ ]:
COMPLEX_PATTERNS = {
    "premises": {
        "all": [
            {"tpl": "所有{a_pl}都是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "所有的{a_pl}都属于{b_pl}的范畴。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "没有例外，所有{a_pl}都是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "所有{a_pl}可以被归类为{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}都被视为{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "每个{a_sg}都是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "凡是{a_sg}，必然是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "{a_pl}整体上等同于{b_pl}的一部分。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "所有的{a_pl}无一例外地属于{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_sg}总是属于{b_sg}的一类。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "只要是{a_sg}，就一定是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "任何一个{a_sg}都是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "全部的{a_pl}都包含在{b_pl}之中。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}是{b_pl}的一个子集。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "凡是符合{a_sg}定义的，均属于{b_sg}。", "a_type": "singular", "b_type": "singular"}
        ],
        "some": [
            {"tpl": "一些{a_pl}是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "有些{a_pl}属于{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "部分{a_pl}可以被认为是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}中有部分是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "存在若干{a_pl}是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "至少有一个{a_sg}是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "有一部分{a_pl}确实是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "并非所有，但有些{a_pl}是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "在{a_pl}中，能找到是{b_pl}的例子。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "有一部分{a_pl}落入{b_pl}的范畴。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}里的某些个体也是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "某些{a_pl}被观察到是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "一小部分{a_pl}属于{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "确实存在一些{a_pl}是{b_pl}的情况。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "有少数{a_pl}是{b_pl}。", "a_type": "plural", "b_type": "plural"}
        ],
        "no": [
            {"tpl": "没有{a_pl}是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "绝对没有{a_pl}属于{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}与{b_pl}没有交集。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "并不存在{a_pl}是{b_pl}的情况。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}完全不是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "没有任何一个{a_sg}是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "凡是{a_sg}，都不可能是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "{a_pl}被完全排除在{b_pl}之外。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}和{b_pl}是互斥的。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "在任何情况下，{a_sg}都不是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "找不到任何是{b_pl}的{a_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_sg}不属于{b_sg}这一类。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "没有哪一个{a_sg}会被归为{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "所有的{a_pl}都不是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_sg}绝对不是{b_sg}。", "a_type": "singular", "b_type": "singular"}
        ],
        "some_not": [
            {"tpl": "一些{a_pl}不是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}中有些并非{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "部分{a_pl}不属于{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "存在某些{a_pl}不是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "并非所有{a_pl}都是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "至少有一个{a_sg}不是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "有一些{a_pl}在{b_pl}的范畴之外。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "有一部分{a_pl}不符合{b_pl}的特征。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "并非每一个{a_sg}都是{b_sg}。", "a_type": "singular", "b_type": "singular"},
            {"tpl": "在{a_pl}中，存在不属于{b_pl}的个体。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "某些{a_pl}并不被视为{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "有一部分{a_pl}被排除在{b_pl}之外。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "{a_pl}里的有些成员不是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "并不保证所有的{a_pl}都是{b_pl}。", "a_type": "plural", "b_type": "plural"},
            {"tpl": "有少数{a_pl}不属于{b_pl}。", "a_type": "plural", "b_type": "plural"}
        ]
    },
    "conclusions": [
        "因此，{conclusion}。",
        "由此可知，{conclusion}。",
        "所以，{conclusion}。",
        "综上，{conclusion}。",
        "因此可以得出结论：{conclusion}。",
        "由上述前提可推断：{conclusion}。",
        "据此，{conclusion}。",
        "结论是，{conclusion}。",
        "归纳来看，{conclusion}。",
        "显而易见，{conclusion}。"
    ]
}

In [ ]:
COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    "premises": {
        "all": [
            {"tpl": "所有是{a_sg}的{d_sg}也都是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "若{d_sg}是{a_sg}，则它是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "没有任何既是{d_sg}又是{a_sg}却不是{b_sg}的情况。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "凡是既为{d_sg}又为{a_sg}的，必然是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "{d_pl}中所有属于{a_pl}的，都属于{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "任何具有{a_sg}属性的{d_sg}都是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "在{d_pl}这个群体中，所有是{a_pl}的也属于{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "只要是{a_sg}的{d_sg}，就一定是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "每个是{a_sg}的{d_sg}都是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "如果某个{d_sg}是{a_sg}，那么它肯定也是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "所有属于{a_pl}范畴的{d_pl}都包含在{b_pl}中。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "凡是{d_sg}且符合{a_sg}条件的，无一例外是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "全体既是{d_pl}又是{a_pl}的成员都是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "在{d_pl}里，所有被视为{a_pl}的都被归类为{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "每一个既是{d_sg}又是{a_sg}的个体都属于{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"}
        ],
        "some": [
            {"tpl": "存在至少一个{d_sg}，它既是{a_sg}也是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "有些{d_pl}同时是{a_pl}且是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "在某些情况下，{d_sg}是{a_sg}且是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "部分{d_pl}既为{a_pl}又为{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "有若干{d_sg}同时具有{a_sg}和{b_sg}的特征。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "在{d_pl}中，有些是{a_pl}同时也是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "至少有一些{d_pl}具有{a_pl}和{b_pl}的双重属性。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "我们可以找到既是{d_sg}又是{a_sg}还是{b_sg}的例子。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "有些{d_pl}落入{a_pl}和{b_pl}的交集。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "某些{d_pl}被观察到既是{a_pl}也是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "在{d_pl}这个范围内，存在一些{a_pl}也是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "确实有一部分{d_pl}是{a_pl}且是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "有少数{d_pl}同时属于{a_pl}和{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "存在某些{d_pl}，它们既表现出{a_pl}的特性，也表现出{b_pl}的特性。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "在{d_pl}中，有一部分成员既属于{a_pl}也属于{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"}
        ],
        "no": [
            {"tpl": "{d_pl}中没有任何属于{a_pl}的同时属于{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "不存在既是{d_sg}又是{a_sg}且是{b_sg}的个体。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "如果{d_sg}是{a_sg}，则不可能是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "没有{d_pl}同时为{a_pl}和{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "凡是{d_sg}且为{a_sg}的，都不属于{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "没有任何一个{d_sg}是既属于{a_sg}又属于{b_sg}的。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "在{d_pl}中，{a_pl}属性和{b_pl}属性是互斥的。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "凡是具有{a_sg}特征的{d_sg}都不是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "根本找不到既是{d_sg}、{a_sg}又是{b_sg}的情况。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "在{d_pl}中，凡是属于{a_pl}的都被排除在{b_pl}之外。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "任何{d_sg}都不能同时被归为{a_sg}和{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "在{d_pl}这个范畴内，{a_pl}和{b_pl}没有共同成员。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "绝对没有{d_pl}既属于{a_pl}也属于{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "没有任何属于{a_pl}范畴的{d_pl}会被认为是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "既是{d_sg}又是{a_sg}的个体绝对不可能是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"}
        ],
        "some_not": [
            {"tpl": "并非所有{d_sg}是{a_sg}的都为{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "至少有一个{d_sg}是{a_sg}但不是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "一些{d_pl}属于{a_pl}但不属于{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "有些{d_sg}虽为{a_sg}，却不是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "部分{d_pl}是{a_pl}但并非{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "在{d_pl}中，有些属于{a_pl}但不落入{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "并不是每一个既是{d_sg}又是{a_sg}的都是{b_sg}。", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
            {"tpl": "有些{d_pl}具有{a_pl}的属性，却不具有{b_pl}的属性。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "在{d_pl}中，存在是{a_pl}但不是{b_pl}的个体。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "某些属于{a_pl}范畴的{d_pl}并不属于{b_pl}范畴。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "并不是所有的{d_pl}中的{a_pl}都能被归为{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "存在一些{d_pl}成员，它们是{a_pl}却被排除在{b_pl}之外。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "并非全体既是{d_pl}又是{a_pl}的成员都是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "有一部分{d_pl}虽然是{a_pl}，但不是{b_pl}。", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
            {"tpl": "有少数{d_pl}既符合{a_pl}特征，又不符合{b_pl}特征。", "a_type": "plural", "b_type": "plural", "d_type": "plural"}
        ]
    }
}

In [ ]:
def get_singular_and_plural(term):
    term = term.strip()
    # For Chinese, singular/plural and articles are not distinct in the same way as English.
    # Return the same term for compatibility with templates.
    return term, term

In [ ]:
def get_article(term):
    # Chinese does not use 'a'/'an' articles; return the term unchanged for templates.
    term = term.strip()
    return term

In [ ]:
def get_basic_syllogism_parts(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    return sentences

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    # Keep English parsing of input simple syllogisms (input remains English symbolic form)
    match = re.match(r"All (.*) that are (.*) are (.*).", sentence)
    if match: return "all", match.group(1), match.group(2), match.group(3)
    match = re.match(r"Some (.*) that are (.*) are not (.*).", sentence)
    if match: return "some_not", match.group(1), match.group(2), match.group(3)
    match = re.match(r"Some (.*) that are (.*) are (.*).", sentence)
    if match: return "some", match.group(1), match.group(2), match.group(3)
    match = re.match(r"No (.*) that are (.*) are (.*).", sentence)
    if match: return "no", match.group(1), match.group(2), match.group(3)
    match = re.match(r"All (.*) are (.*).", sentence)
    if match: return "all", None, match.group(1), match.group(2)
    match = re.match(r"Some (.*) are not (.*).", sentence)
    if match: return "some_not", None, match.group(1), match.group(2)
    match = re.match(r"Some (.*) are (.*).", sentence)
    if match: return "some", None, match.group(1), match.group(2)
    match = re.match(r"No (.*) are (.*).", sentence)
    if match: return "no", None, match.group(1), match.group(2)
    raise ValueError(f"Could not parse basic syllogism type from sentence: {sentence}")

In [ ]:
def split_syllogism_into_sentences(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    if len(sentences) < 2:
        raise ValueError(f"Syllogism must contain at least one premise and one conclusion: {syllogism}")
    sentences = [s + '.' for s in sentences]
    conclusion = sentences.pop()
    premises = sentences
    return premises, conclusion

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth, synonym_dict=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    premises_simple, conclusion_simple = split_syllogism_into_sentences(simple_syllogism)
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        try:
            p_complex = p_complex[0].upper() + p_complex[1:]
        except Exception:
            pass
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    # remove last char (punctuation) and attempt to capitalize first char if applicable
    c_complex = c_complex[:-1]
    try:
        c_complex = c_complex[0].upper() + c_complex[1:]
    except Exception:
        pass
    final_complex_syllogism = " ".join(complex_premises) + " " + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f" Skipping item ID {item.get('id', 'N/A')} due to parsing error: {e}")
            continue
    return complex_dataset

In [ ]:
# Example Data for Testing (input syllogisms remain in English/simple symbolic form)
sample_data = [
    {"id": "example-1", "syllogism": "All animals that are non-aquatic are fish. Some mammals are non-aquatic. Some mammals are not fish.", "validity": True, "plausibility": True},
    {"id": "example-2", "syllogism": "All υ are Π. All Π are Ε. Some Ε are υ.", "validity": True, "plausibility": True},
    {"id": "example-3", "syllogism": "No fish are mammals. Some mammals are whales. Some whales are not fish.", "validity": True, "plausibility": True},
]
complex_syllogism_data = process_dataset(sample_data, COMPLEX_PATTERNS, COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print("
--- 转换后的复杂三段论示例 ---")
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))